### S001 Basic RAG ###
- Create chunks, embed, cross your fingers 
- Supported strategies
    - S001_00 -> Basic RAG
    - S001_01 -> Basic RAG + Rereank

In [ ]:
import os
from config import set_environment 
set_environment()

import logging
import sys
#logging.basicConfig(stream=sys.stdout, level=logging.INFO)
#logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

# Only for notebook
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from llama_index.core import Settings

from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import VectorStoreIndex
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor, KeywordNodePostprocessor
from llama_index.postprocessor.cohere_rerank import CohereRerank

from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

from llama_index.llms.cohere import Cohere
from llama_index.embeddings.cohere import CohereEmbedding

from llama_index.llms.anthropic import Anthropic
from llama_index.llms.together import TogetherLLM

from llama_index.llms.gemini import Gemini
from llama_index.embeddings.gemini import GeminiEmbedding

from datetime import datetime
import pandas as pd
import random


Choose the model for generation

In [ ]:
generation_llm_family = os.environ["GENERATION_LLM_FAMILY"]
generation_llm_model = os.environ["GENERATION_LLM_MODEL"]

if generation_llm_family == "OPENAI":
    Settings.llm = OpenAI(temperature=0, model=generation_llm_model)
elif generation_llm_family == "COHERE":
    Settings.llm = Cohere(api_key=os.environ["COHERE_API_KEY"], model=generation_llm_model,temperature=0)
elif generation_llm_family == "ANTHROPIC":
    Settings.llm = Anthropic(model=generation_llm_model, temperature=0)
elif generation_llm_family == "META" or generation_llm_family == "QWEN":
    Settings.llm = TogetherLLM(model=generation_llm_model, api_key=os.environ["TOGETHER_API_KEY"]
)
elif generation_llm_family == "GOOGLE":
    Settings.llm = Gemini(model=generation_llm_model,temperature=0)

Choose the model for embedding

In [ ]:
embedding_llm_family = os.environ["EMBEDDING_LLM_FAMILY"]
embedding_llm_model = os.environ["EMBEDDING_LLM_MODEL"]
embedding_dimensions = int(os.environ["EMBEDDING_DIMESIONS"])

if embedding_llm_family == "OPENAI":
    Settings.embed_model = OpenAIEmbedding(model=embedding_llm_model,dimensions=embedding_dimensions,)
elif embedding_llm_family == "COHERE":
    Settings.embed_model = CohereEmbedding(
    cohere_api_key=os.environ["COHERE_API_KEY"],
    model_name=embedding_llm_model,
    input_type="search_query",
)
elif embedding_llm_family == "GOOGLE":
    Settings.embed_model = GeminiEmbedding(
        model_name=embedding_llm_model, api_key=os.environ["GOOGLE_API_KEY"]
)

Set the parameters for the run here

In [ ]:
eval_name = os.environ["EVAL_NAME"]
eval_directory = os.environ["EVAL_DIRECTORY"]
eval_file = os.environ["EVAL_FILE"]
eval_questions = os.environ["EVAL_QUESTIONS"]
eval_results_dir = os.environ["EVAL_RESULTS_DIR"]
eval_quick_test = os.environ["EVAL_QUICK_TEST"]

rag_strategy = os.environ["RAG_STRATEGY"]

similarity_top_k = int(os.environ["SIMILARITY_TOP_K"])

# Context Post Processor Settings
similarity_cutoff = float(os.environ["SIMILARITY_CUTOFF"])

# Node Parser
chunk_size = int(os.environ["CHUNK_SIZE"])
chunk_overlap = 0.1 * chunk_size

Pick the strategy

In [ ]:
generation_string = generation_llm_model.replace("meta-llama/", "").replace("Qwen/", "").replace("models/", "") #if "meta-llama/" in generation_llm_model else generation_llm_model
embed_string = embedding_llm_model.replace("models/", "") if "models/" in embedding_llm_model else embedding_llm_model

if rag_strategy == "S001_00":
    rag_strategy_desc = "Basic"
    batch_id = f"{eval_name}_{rag_strategy}_GM_{generation_string}_EM_{embed_string}_C_{chunk_size}_K_{similarity_top_k}_{random.randint(0, 999):03}"
elif rag_strategy == "S001_01": 
    rag_strategy_desc = "Basic_Rerank"
    reranker = os.environ["RERANKER"]
    rerank_top_n = int(os.environ["RERANK_TOP_N"])
    batch_id = f"{eval_name}_{rag_strategy}_GM_{generation_string}_EM_{embed_string}_C_{chunk_size}_K_{similarity_top_k}_RR_{reranker}_N_{rerank_top_n}_{random.randint(0, 999):03}"

output_file = f"{eval_results_dir}/{batch_id}.xlsx"

Set up Token Counting

In [ ]:
import tiktoken
from llama_index.core.callbacks import CallbackManager, TokenCountingHandler

token_counter = TokenCountingHandler(
    tokenizer=tiktoken.encoding_for_model("gpt-4").encode
)

Settings.callback_manager = CallbackManager([token_counter])
tokencount_df = pd.DataFrame()

Read the documents, create chunks, calculate embeddings, store in a vector database

In [ ]:
reader = SimpleDirectoryReader(eval_directory)
documents = reader.load_data()

node_parser = SentenceSplitter(chunk_size=chunk_size, chunk_overlap = chunk_overlap)
nodes = node_parser.get_nodes_from_documents(documents)
# set node ids to be a constant
for idx, node in enumerate(nodes):
    node.id_ = f"node-{idx}"

index = VectorStoreIndex(nodes, embed_model=Settings.embed_model, show_progress=True)

In [ ]:
tokencount_df['document_tokens'] = [token_counter.total_embedding_token_count]
token_counter.reset_counts()

Set up retrieval and response generation

In [ ]:
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=similarity_top_k
)

if rag_strategy =="S001_00":
    node_postprocessors = [
        SimilarityPostprocessor(similarity_cutoff=similarity_cutoff) 
    ]
elif rag_strategy == "S001_01":
    cohere_rerank = CohereRerank(api_key=os.environ["COHERE_API_KEY"], top_n=rerank_top_n)
    node_postprocessors = [
        SimilarityPostprocessor(similarity_cutoff=similarity_cutoff), cohere_rerank
    ]


# This is the most basic type of response generation. Send the retrieved chunks to the LLM and display the receieved response

query_engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=node_postprocessors
)

Quick test of query engine

In [ ]:
#eval_quick_test = """I was in an accident. I worked on Monday and Tue. I took Wed, Thu, Fri off. 
#The doctor agrees I need to go on disability. Can we start STD next Monday. Think step by step. ? 

#"""

In [ ]:
response = query_engine.query(eval_quick_test)
print(f"Question:{eval_quick_test}{chr(10)}")
print(f"Response:{chr(10)}{response.response}{chr(10)}")

text_md = ""
for n in response.source_nodes:
    
    text_md += (
        f"**Node ID:** {n.node.node_id}{chr(10)}"
        f"**Similarity:** {n.score}{chr(10)}"
        f"**Text:** {n.node.get_content()}{chr(10)}"
        f"**Metadata:** {n.node.metadata}{chr(10)}"
        f"~~~~{chr(10)}"
    )

print(f"Source Nodes:{chr(10)}")
print(text_md)

- Read the evalution question set (along with expected answers)
- This is structured in Llamaindex's format for batch evaluations
- Also, load into a data frame (which we will write back to an excel file with responses, evaluations etc.)

In [ ]:
with open(eval_questions, 'r') as file:
    data = pd.read_json(file)
     
    queries_df = pd.DataFrame(list(data['queries'].items()), columns=['query_num', 'query'])
    responses_df = pd.DataFrame(list(data['responses'].items()), columns=['query_num', 'expected_answer'])
    
    responses_df = pd.merge(queries_df, responses_df, on='query_num')

In [ ]:
from llama_index.core.evaluation.eval_utils import (
    get_responses,
)
from llama_index.core.evaluation import QueryResponseDataset

Send questions to engine in bulk

In [ ]:
eval_dataset = QueryResponseDataset.from_json(eval_questions)
eval_qs = eval_dataset.questions
ref_response_strs = [r for (_, r) in eval_dataset.qr_pairs]
pred_responses = get_responses(
    eval_qs, query_engine, show_progress=True
)
pred_response_strs = [str(p) for p in pred_responses]

In [ ]:
from evaluation_utils import get_eval_results_df, get_summary_scores_df, get_answers_source_nodes

answers, sources = get_answers_source_nodes(pred_responses)

responses_df['generated_answer'] = answers

sources_df = pd.DataFrame()
sources_df['query_num'] = responses_df['query_num']
sources_df['query'] = responses_df['query']
sources_df = sources_df.join(pd.DataFrame(sources)[0].str.split("~~~~", expand=True))
sources_df['batch_id'] = batch_id

tokencount_df['answer_tokens' ] = [token_counter.total_llm_token_count]
token_counter.reset_counts()

In [ ]:
with pd.ExcelWriter(output_file) as writer:
   responses_df.to_excel(writer, sheet_name="Responses", index=False)
   sources_df.to_excel(writer, sheet_name="Sources", index=False)
   tokencount_df.to_excel(writer, sheet_name="Token Counts", index=False)

Choose the LLM for evaluations

In [ ]:
evaluation_llm_family = os.environ["EVALUATION_LLM_FAMILY"]
evaluation_llm_model = os.environ["EVALUATION_LLM_MODEL"]

if evaluation_llm_family == "OPENAI":
    Settings.eval_llm = OpenAI(temperature=0, model=evaluation_llm_model)
elif evaluation_llm_family == "COHERE":
    Settings.eval_llm = Cohere(api_key=os.environ["COHERE_API_KEY"], model=evaluation_llm_model, temperature=0)

Setup Evaluations

In [ ]:
from llama_index.core.evaluation import QueryResponseDataset
from llama_index.core.evaluation.eval_utils import (
    get_responses,
)
from llama_index.core.evaluation import BatchEvalRunner

from llama_index.core.evaluation import (
    CorrectnessEvaluator,
)
from deepeval.integrations.llama_index import (
    DeepEvalAnswerRelevancyEvaluator,
    DeepEvalFaithfulnessEvaluator,
    DeepEvalContextualRelevancyEvaluator,
    DeepEvalBiasEvaluator,
    DeepEvalToxicityEvaluator,
)

eval_lidx_c = CorrectnessEvaluator(llm=Settings.eval_llm)
eval_deval_f = DeepEvalFaithfulnessEvaluator(threshold=0.5, model=evaluation_llm_model,include_reason=True)
eval_deval_ar = DeepEvalAnswerRelevancyEvaluator( threshold=0.5, model=evaluation_llm_model,include_reason=True)
eval_deval_cr = DeepEvalContextualRelevancyEvaluator(threshold=0.5, model=evaluation_llm_model,include_reason=True)
eval_deval_b = DeepEvalBiasEvaluator(threshold=0.5, model=evaluation_llm_model,include_reason=True)
eval_deval_t = DeepEvalToxicityEvaluator(threshold=0.5, model=evaluation_llm_model,include_reason=True)

In [ ]:
# For large eval sets (30+ questions)
evaluator_dict_essential = {
    "Correctness": eval_lidx_c
}

# For troubleshooting 
evaluator_dict_extended = {
    "Correctness": eval_lidx_c,
    "Faithfulness": eval_deval_f,
    "Context_Relevancy": eval_deval_cr
}

# For small sets (< 10 questions)
evaluator_dict_full = {
    "Correctness": eval_lidx_c,
    "Faithfulness": eval_deval_f,
    "Answer_Relevancy": eval_deval_ar,
    "Context_Relevancy": eval_deval_cr,
    "Bias": eval_deval_b,
    "Toxicity": eval_deval_t ,
}

# Pick the list of evaluators to run
evaluator_dict = evaluator_dict_essential

# Make sure this list matches the chosenevaluator_dict 
evaluators = ["Correctness" ] 

batch_runner = BatchEvalRunner(evaluator_dict, workers=8)

In [ ]:
eval_results = await batch_runner.aevaluate_responses(
    queries=eval_qs,
    responses=pred_responses,
    reference=ref_response_strs,
)

In [ ]:
from evaluation_utils import get_eval_results_df, get_summary_scores_df

In [ ]:
mean_df, sum_df = get_summary_scores_df(
    [eval_results ],
    [rag_strategy],
    evaluators
)

In [ ]:
if "Correctness" in evaluators:
    correctness_df = get_eval_results_df(
        list(responses_df['query_num']),
        list(responses_df['expected_answer']),
        eval_results["Correctness"]
    )
    correctness_df.rename(columns={'score': 'correctness_llm'}, inplace=True)
    correctness_df.rename(columns={'feedback': 'feedback_llm'}, inplace=True)
    correctness_df['correctness_human'] = correctness_df['correctness_llm']
    correctness_df['feedback_human'] = ""
    correctness_df['batch_id'] = batch_id
    
    responses_df['correctness_llm'] = correctness_df['correctness_llm']
    responses_df['correctness_human'] = correctness_df['correctness_human']

if "Faithfulness" in evaluators:
    faithfulness_df = get_eval_results_df(
        list(responses_df['query_num']),
        list(responses_df['expected_answer']),
        eval_results["Faithfulness"]
    )
    
    faithfulness_df.rename(columns={'score': 'faithfulness_llm'}, inplace=True)
    faithfulness_df.rename(columns={'feedback': 'feedback_llm'}, inplace=True)
    faithfulness_df['faithfulness_human'] = faithfulness_df['faithfulness_llm']
    faithfulness_df['feedback_human'] = ""
    faithfulness_df['batch_id'] = batch_id
    
    responses_df['faithfulness_llm'] = faithfulness_df['faithfulness_llm']
    responses_df['faithfulness_human'] = faithfulness_df['faithfulness_human']

if "Answer_Relevancy" in evaluators:
    answer_relevancy_df = get_eval_results_df(
        list(responses_df['query_num']),
        list(responses_df['expected_answer']),
        eval_results["Answer_Relevancy"]
    )
    responses_df['answer_relevancy'] = answer_relevancy_df['score']

if "Context_Relevancy" in evaluators:
    context_relevancy_df = get_eval_results_df(
        list(responses_df['query_num']),
        list(responses_df['expected_answer']),
        eval_results["Context_Relevancy"]
    )
    responses_df['context_relevancy'] = context_relevancy_df['score']

if "Bias" in evaluators:
    bias_df = get_eval_results_df(
        list(responses_df['query_num']),
        list(responses_df['expected_answer']),
        eval_results["Bias"]
    )
    responses_df['bias'] = bias_df['score']

if "Toxicity" in evaluators:
    toxicity_df = get_eval_results_df(
        list(responses_df['query_num']),
        list(responses_df['expected_answer']),
        eval_results["Toxicity"]
    )
    responses_df['toxicity'] = toxicity_df['score']



In [ ]:
responses_df['rag_strategy'] = rag_strategy
responses_df['rag_strategy_desc'] = rag_strategy_desc
responses_df['parameter_1'] = chunk_size
responses_df['parameter_2'] = similarity_top_k
if rag_strategy =="S001_00":
   responses_df['parameter_3'] = ""
elif rag_strategy == "S001_01":
    responses_df['parameter_3'] = rerank_top_n
responses_df['parameter_4'] = ""
responses_df['parameter_5'] = ""
responses_df['model'] = generation_llm_model 
responses_df['embed_model'] = embedding_llm_model 
responses_df['eval_model'] = evaluation_llm_model
responses_df['embed_dimensions'] = embedding_dimensions
if rag_strategy =="S001_00":
   responses_df['reranker'] = ""
elif rag_strategy == "S001_01":
    responses_df['reranker'] = reranker
responses_df['run_date'] = datetime.today().strftime('%Y-%m-%d') 
responses_df['eval_name'] = eval_name
responses_df['batch_id'] = batch_id

In [ ]:
columns_to_set_none = [
    'total_tokens',
    'prompt_tokens',
    'completion_tokens',
    'total_cost',
    'prompt_cost',
    'completion_cost',
    'latency',
    'first_token_ms'
]

# Set the specified columns to None
responses_df.loc[:, columns_to_set_none] = None

In [ ]:
tokencount_df['eval_tokens' ] = [token_counter.total_llm_token_count]
token_counter.reset_counts()

In [ ]:
with pd.ExcelWriter(output_file) as writer:
   responses_df.to_excel(writer, sheet_name="Responses", index=False)
   sources_df.to_excel(writer, sheet_name="Sources", index=False)
   
   sum_df.to_excel(writer, sheet_name="Summary", index=False, startrow=0 , startcol=0)
   mean_df.to_excel(writer, sheet_name="Summary", index=False,startrow=5, startcol=0)
   
  
   if "Correctness" in evaluators:
      correctness_df.to_excel(writer, sheet_name="Correctness", index=False)
   
   if "Faithfulness" in evaluators:
      faithfulness_df.to_excel(writer, sheet_name="Faithfulness", index=False)

   if "Context_Relevancy" in evaluators:
      context_relevancy_df.to_excel(writer, sheet_name="Context_Relevancy", index=False)
   
   if "Answer_Relevancy" in evaluators:
      answer_relevancy_df.to_excel(writer, sheet_name="Answer_Relevancy", index=False)
   
   if "Bias" in evaluators:
      bias_df.to_excel(writer, sheet_name="Bias", index=False)
   
   if "Toxicity" in evaluators:
      toxicity_df.to_excel(writer, sheet_name="Toxicity", index=False)
   
   tokencount_df.to_excel(writer, sheet_name="Token Counts", index=False)
